In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Tải dữ liệu từ file CSV
data = pd.read_csv('_Dữ_liệu_giao_dịch_ngày__202406152152.csv')

In [2]:
data.drop(['Ghép 01','Ghép 02','Ngành nghề'], axis=1, inplace= True)

In [3]:


# Thêm cột 'profit' là lợi nhuận dự đoán từ giá mở cửa đến giá đóng cửa
data['profit'] = data['Đóng cửa'] - data['Mở cửa']

# Chọn các đặc trưng và nhãn
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất', 'Trung bình', 'GD khớp lệnh KL']
data = data.set_index('Ngày')
data = data.sort_index()

# Tiền xử lý dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

# Chuyển đổi dữ liệu thành dạng chuỗi thời gian cho LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng và huấn luyện mô hình
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                   return_sequences=True, 
                   input_shape=(seq_length, len(features))))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                   return_sequences=False))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=1))
    
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    
    return model

tuner = kt.RandomSearch(build_model,
                        objective='val_loss',
                        max_trials=3,
                        executions_per_trial=1,
                        directory='tuner_results',
                        project_name='stock_profit_prediction')

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
best_model = tuner.get_best_models(num_models=1)[0]

# Lưu mô hình đã huấn luyện
with open('best_lstm_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

Trial 3 Complete [03h 43m 18s]
val_loss: 0.005270658526569605

Best val_loss So Far: 0.005270658526569605
Total elapsed time: 16h 07m 00s


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
